# Platform information

In [1]:
library(benchmarkme)
get_platform_info()$OS.type
get_r_version()$version.string
get_cpu()$model_name;get_cpu()$no_of_cores
get_ram()

[1] "windows"

[1] "R version 3.5.1 (2018-07-02)"

[1] "Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz"

[1] 8

8.59 GB

# Parallel computation

In [2]:
library(doParallel)
library(foreach)
cl<-makeCluster(8) 

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel


# Loading data

In [3]:
geno <-read.table("./Hetero_realigned_cov10_filtered3.raw", row.names=1, header=T)
row.names(geno)=1:240
x <- as.matrix(geno)-1   
pheno <- read.csv("./2017heteroPheno.csv", header=T)
attach(pheno)
data <- data.frame(bcw=pheno$bcw,length=pheno$length,gid=1:240)

In [4]:
rmd.x<-x[,!duplicated.matrix(x,MARGIN = 2)]

# Parameters for cross validation

In [5]:
repeats <- 10
n.fold <- 5 
n.sample <- length(pheno$bcw)    
CM<-7

# Bayes cpi

I found the code from this lab
 http://www.zzlab.net/StaGen/

In [5]:
source("http://www.zzlab.net/sandbox/BAGS.R")

In [8]:
registerDoParallel(cl)
system.time({
Cpi<-foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold) %dopar% {
         bcw_test <- data
         m1<-bcw_test$bcw[id != i] 
         u1<-as.matrix(geno)[id != i, ]        
         ki<-BAGS(u1, y=m1,pi=1,burn.in=1000,burn.out=2000)
         pred<-(as.matrix(geno)[id == i, ]) %*% ki$effect
         cor(pheno$bcw[id==i],pred)
    }
   }
})
stopImplicitCluster()

    user   system  elapsed 
    1.36     2.53 14754.27 

In [18]:
Cpi

result.1,0.2219468,0.1594255,0.1886464,0.3257046,0.008631627
result.2,0.04603167,0.329667,0.1702699,0.05681837,0.2823873
result.3,0.1618135,0.3293952,-0.08750843,0.1775165,-0.01163318
result.4,0.3945332,0.4198629,0.02603833,0.01668889,0.1229961
result.5,0.3297482,0.1469074,0.1622235,0.07863016,0.3126024
result.6,0.3477457,0.3905192,0.0518323,0.07439348,0.4505462
result.7,0.3303076,0.3233216,0.1655145,0.1465404,0.4664679
result.8,0.3482255,0.1964356,0.07313449,0.2668536,0.2184256
result.9,0.1701746,0.05899124,0.2461707,-0.09392551,0.2994342
result.10,0.1620705,0.1675907,0.1824422,0.2914718,0.1193513
